# Torchmeter

An `all-in-one` tool for `Pytorch` model analysis, measuring:
- Params,
- FLOPs / MACs (aka. MACC or MADD), 
- Memory cost, 
- Inference time
- Throughput

Project: https://github.com/Ahzyuan/torchmeter

In [ ]:
# Note: the `patest` here is a simple package created by myself
#       that only use for this demo.
%pip install torchmeter

In [ ]:
import torch
from rich import print
from torchvision import models
from torchmeter import Meter, get_config

# 1. 模型准备与Meter初始化（展示零侵入代理）
model = models.vgg19_bn()
metered_model = Meter(model)

# 2. 输入处理与推理（展示设备同步）
input = torch.randn(1, 3, 224, 224)
if torch.cuda.is_available():
    input = input.to('cuda')
output = metered_model(input)  # 标准前向传播




    


In [ ]:
import torch
from rich import print
from torchvision import models
from torchmeter import Meter, get_config

# 初始化配置（展示配置驱动管理）
config = get_config()
config.tree_fold_repeat = True  # 启用重复模块折叠
config.table_display_args.style = "cyan"  # 全局表格样式

def main():
    # 1. 模型准备与Meter初始化（展示零侵入代理）
    model = models.resnet18()
    meter = Meter(model, device='cuda')  # 自动检测设备并同步
    
    # 2. 输入处理与推理（展示设备同步）
    input_tensor = torch.randn(1, 3, 224, 224).cuda()
    output = meter(input_tensor, some_flag=True)  # 标准前向传播
    
    # 3. 性能分析（展示全栈分析能力）
    # --------------------------------------------
    # 3.1 参数分析（展示参数测量）
    print("[bold cyan]\n=== 参数分析 ===")
    print(meter.param)  # 总参数统计
    meter.profile('param', no_tree=True)  # 分层参数分布
    
    # 3.2 计算分析（展示FLOPs/MACs）
    print("[bold cyan]\n=== 计算分析 ===")
    print(meter.cal)  # 总计算成本
    meter.profile('cal', pick_cols=['Operation_Name', 'FLOPs', 'MACs'])
    
    # 3.3 内存分析（展示内存诊断）
    print("[bold cyan]\n=== 内存分析 ===")
    print(meter.mem)
    meter.profile('mem', 
                custom_cols={'Operation_Name': '模块名称'},
                save_to='memory_report.csv')
    
    # 3.4 性能基准测试（展示延迟和吞吐量）
    print("[bold cyan]\n=== 性能基准 ===")
    meter.ittp_warmup = 100  # 定制预热次数
    print(meter.ittp)  # 获取最新基准数据
    
    # 4. 可视化功能（展示实时渲染与配置）
    # --------------------------------------------
    # 4.1 结构树可视化（展示智能折叠）
    print("[bold cyan]\n=== 模型结构树 ===")
    meter.tree_levels_args = {"1": {"guide_style": "bold yellow"}}  # 动态配置
    print(meter.structure)
    
    # 4.2 交互式表格（展示动态调整）
    print("[bold cyan]\n=== 定制化表格 ===")
    meter.table_display_args.update({
        "show_lines": True,
        "row_styles": ["dim", ""]
    })
    meter.profile('cal', no_tree=True)
    
    # 5. 高级功能演示
    # --------------------------------------------
    # 5.1 设备管理（展示跨平台支持）
    meter.to('cpu')
    meter(input_tensor.cpu())  # 自动同步新设备
    
    # 5.2 节点分析（展示分层诊断）
    print("[bold cyan]\n=== 特定层分析 ===")
    conv_layer = meter.rebase("5")  # 定位到第5层
    print(conv_layer.param)
    
    # 5.3 综合报告（展示数据解耦）
    print("[bold cyan]\n=== 综合报告 ===")
    report = meter.overview('param', 'cal', 'mem', show_warning=False)
    print(report)
    
    # 5.4 配置导出（展示便携预设）
    config.export('profile_settings.yaml')  # 保存当前配置

if __name__ == "__main__":
    main()